### Libraries

In [1]:
import pandas as pd
from kafka import KafkaProducer, KafkaConsumer
from time import sleep
import time
from json import dumps
import json
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import matplotlib.pyplot as plt


### Web Scraping and initialising the producer

In [15]:
producer = KafkaProducer(
    bootstrap_servers=[':9092'], # add your public ip which is there in the instance
    value_serializer=lambda x: dumps(x).encode('utf-8')
)


ticker = 'RELIANCE'
url = f'https://www.google.com/finance/quote/{ticker}:NSE?hl=en'

for i in range(5):  
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    price_element = soup.find('div', class_='YMlKec fxKbKc')
    
    if price_element:
        stock_price = float(price_element.text.strip()[1:].replace(',', ''))
        current_time = datetime.now().strftime('%H:%M:%S')
        stock_data = {'Time': current_time, 'Price': stock_price}

        # Sending stock data to Kafka
        producer.send('demo_test1', value=stock_data)
        print(f"Sent data to Kafka: {stock_data}")

    else:
        print("Unable to fetch stock price. Check the HTML structure.")
    
    time.sleep(2)  

Sent data to Kafka: {'Time': '18:40:07', 'Price': 1221.5}
Sent data to Kafka: {'Time': '18:40:13', 'Price': 1221.5}
Sent data to Kafka: {'Time': '18:40:19', 'Price': 1221.5}
Sent data to Kafka: {'Time': '18:40:25', 'Price': 1221.5}
Sent data to Kafka: {'Time': '18:40:31', 'Price': 1221.5}


In [14]:
producer.flush()